# BigQuery Chat with Ethereum

This Jupyter notebook allows you to chat with the public Ethereum dataset on BigQuery using natural language. It uses a Gradio UI to provide a user interface for asking questions and viewing the results.

## Setup

1.  **Install Dependencies:** Follow the readme file of this repo. It will install all the necessary Python libraries using `uv`.
2.  **Configure Environment Variables:** Make sure the `.env` file is in the root of this project and add complete variables:
    ```
    GOOGLE_CLOUD_PROJECT=<your-gcp-project-id>
    GOOGLE_CLOUD_LOCATION=<your-gcp-location>
    ```
3.  **Authentication:** Make sure you are authenticated with Google Cloud. You can do this by running `gcloud auth application-default login` in your terminal.

## Execution

1.  **Run all cells:** Run all the cells in this notebook to start the Gradio UI.
2.  **Ask a question:** Use the textbox to ask a question about the Ethereum dataset (e.g., "Show the 10 most recent transactions").
3.  **View the results:** The generated SQL query, the results of the query, and any logs will be displayed in the output textboxes.


## Dependencies

In [ ]:
import sys
from pathlib import Path
parent_dir = Path("..").resolve()
if str(parent_dir) not in sys.path:
	sys.path.insert(0, str(parent_dir))
	print(f"Added {parent_dir} to Python path.")
else:
	print(f"{parent_dir} already in Python path.")

In [ ]:
# Set option for Pandas without truncating long strings

import pandas as pd
pd.set_option("display.max_colwidth", None)   # don’t truncate long strings
pd.set_option("display.max_columns", None)    # show all columns
pd.set_option("display.width", 0)             # auto-detect console width

## BigQuery Configuration

In [ ]:
# Create a BigQuery client using environment variables, with Colab auth if needed.
import os, re, sys
from dotenv import load_dotenv
from google.cloud import bigquery
from google.auth.exceptions import DefaultCredentialsError

load_dotenv()

def _clean_env(val: str | None, default: str = "") -> str:
    if val is None:
        return default
    v = val.strip()
    # strip wrapping quotes (either single or double)
    if len(v) >= 2 and v[0] == v[-1] and ord(v[0]) in (39, 34):
        v = v[1:-1]
    # drop inline comments
    if "#" in v:
        v = v.split("#", 1)[0].strip()
    return v

# Prefer official names, then legacy fallbacks
PROJECT_ID = _clean_env(os.getenv("GOOGLE_CLOUD_PROJECT") or os.getenv("PROJECT_ID"), "your-gcp-project-id")
LOCATION  = _clean_env(os.getenv("GOOGLE_CLOUD_LOCATION") or os.getenv("LOCATION"), "US")

# Detect Colab and authenticate if necessary
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    try:
        from google.colab import auth as colab_auth  # type: ignore
        print("Running in Colab: prompting for authentication...")
        colab_auth.authenticate_user()
        print("Colab authentication completed.")
    except Exception as e:
        print("Colab auth not available:", e)

# Basic validation for project id
if not re.fullmatch(r"[a-z][a-z0-9-]{4,61}[a-z0-9]", PROJECT_ID):
    print(f"Warning: PROJECT_ID looks invalid: {PROJECT_ID}")
    print("Update your .env, e.g.: GOOGLE_CLOUD_PROJECT=my-gcp-project")

# Tip: Authenticate with Google Cloud before running (choose one):
# 1) In Colab: the cell above already handled this.
# 2) Locally: gcloud auth application-default login
# 3) Or set GOOGLE_APPLICATION_CREDENTIALS to a service account JSON key path

try:
    client = bigquery.Client(project=PROJECT_ID, location=LOCATION)
    print(f"BigQuery client created for project: {client.project} in {LOCATION}")
except DefaultCredentialsError:
    print("Google Application Default Credentials not found.")
    print("Run: gcloud auth application-default login")
    print("Or set GOOGLE_APPLICATION_CREDENTIALS to a service account key JSON file path.")
    client = None
except Exception as e:
    print("Failed to create BigQuery client:", e)
    client = None


In [ ]:
# Create a writable dataset and a remote LLM model using a BigQuery Connection.
from google.cloud import bigquery
from google.api_core import exceptions as gax_exceptions

# Writable dataset in your project (models/tables go here)
dataset_id = f"{PROJECT_ID}.bq_llm"
dataset = bigquery.Dataset(dataset_id)
dataset.location = LOCATION
try:
    client.create_dataset(dataset, exists_ok=True)
    print(f"Dataset ensured: {dataset_id} ({LOCATION})")
except Exception as e:
    print("Failed to create/ensure dataset:", e)

# Connection resource (edit name if your connection differs)
connection_name = "bq-llm-connection"
connection_resource = f"projects/{PROJECT_ID}/locations/{LOCATION}/connections/{connection_name}"
print("Using connection:", connection_resource)

# Heads-up: public dataset bigquery-public-data.crypto_ethereum.* is in US region.
if LOCATION.upper() != "US":
    print("Warning: LOCATION is not US. Reading US public data with a non-US model/connection may cause cross-region errors.")
    print("Set GOOGLE_CLOUD_LOCATION=US in your .env or copy data into a dataset in your region.")

# Create (or replace) the remote LLM model in your project
summarizer_query = f"""
CREATE OR REPLACE MODEL `{dataset_id}.transaction_summarizer`
REMOTE WITH CONNECTION `{connection_resource}`
OPTIONS (
  remote_service_type = 'CLOUD_AI_LARGE_LANGUAGE_MODEL_V1',
  endpoint = 'gemini-2.5-pro'
);
"""

model_ready = False
try:
    client.query(summarizer_query).result()
    print("Remote LLM model ensured:", f"{dataset_id}.transaction_summarizer")
    model_ready = True
except gax_exceptions.NotFound as e:
    print("Connection not found:", connection_resource)
    print("Tip: Create a BigQuery connection named 'bq-llm-connection' in this location and grant it permissions.")
except gax_exceptions.Forbidden as e:
    print("Permission error when creating model. Ensure you have BigQuery Admin and connection access.")
    print(str(e))
except Exception as e:
    print("Failed to create remote model:", e)


## Troubleshooting

- **`DefaultCredentialsError`**: In Colab, re-run the **Auth** cell. Locally, run `gcloud auth application-default login` or set `GOOGLE_APPLICATION_CREDENTIALS` to a service-account JSON.
- **`NotFound: Connection not found`**: Create a BigQuery **Connection** named `bq-llm-connection` in the `GOOGLE_CLOUD_LOCATION` region, and grant the notebook's identity permission to use it.
- **Cross-region errors**: The public dataset `bigquery-public-data.crypto_ethereum.*` is in the **US** multi-region. Set `GOOGLE_CLOUD_LOCATION=US` (or stage the data to your region).
- **Permission denied**: Ensure your identity has BigQuery Admin (or sufficient) permissions **and** connection usage rights.


In [ ]:
# Optional cleanup: set CLEANUP=True to drop the model and/or dataset
CLEANUP = False

if CLEANUP and client is not None:
    try:
        client.delete_model(f"{dataset_id}.transaction_summarizer", not_found_ok=True)
        print("Deleted model:", f"{dataset_id}.transaction_summarizer")
    except Exception as e:
        print("Model delete failed:", e)
    try:
        client.delete_dataset(dataset_id, delete_contents=True, not_found_ok=True)
        print("Deleted dataset:", dataset_id)
    except Exception as e:
        print("Dataset delete failed:", e)
else:
    print("Cleanup skipped.")


## The AI Architect

- Purpose: Translate natural language into safe, parameterized SQL over BigQuery’s public Ethereum dataset, execute it, and explain results.
- Config: project `{PROJECT_ID}`, location `{LOCATION}`.
- Remote LLM model: `{dataset_id}.transaction_summarizer` (gemini-2.5-pro) via `{connection_resource}` — ready: {model_ready}.
- What you get:
	- Generated SQL preview
	- Query results table
	- Optional LLM summary of results
	- Execution logs/errors
- Safety:
	- Read-only queries against `bigquery-public-data.crypto_ethereum.*`
	- Auto LIMITs for large scans (tunable in UI)
	- Note: public dataset is in US; cross-region reads can error if LOCATION != US
- How to use:
	1) Run the UI cell with `phase = "architect"`.
	2) Ask a question (e.g., “Show the 10 most recent transactions”).
- Example prompts:
	- "List the latest 10 transactions."
	- "Average gas price by day over the past week."

In [ ]:
# Launch the UI with phase selection
from bigq_ai_app.ui import create_ui

# Choose phase: architect, semantic, multimodal
phase = "architect" 

demo = create_ui(phase)
demo.launch()

## The AI Semantic

- Purpose: Semantic retrieval over your ingested docs (RAG). It embeds queries with `{model_fqn}` and finds nearest chunks in `{table_fqn}` using BigQuery VECTOR_SEARCH, then optionally summarizes.
- Config: project `{project_id}`, location `{location}`.
- Data/Models:
	- Docs table: `{table_fqn}` (columns like source_file, kind, content, embedding)
	- Embedding model: `{model_fqn}` (endpoint `text-embedding-004` via `{conn_res}`)
	- Optional vector index: `semantic_docs_index` on the embedding column
- What you get:
	- Top-k matches with content previews and distances
	- (Optional) LLM summary grounded on retrieved chunks
	- Execution logs/errors
- Safety:
	- Read-only queries against your docs table
	- Uses brute-force search by default for small tables; switch to index when created
- How to use:
	1) Ingest docs (run the ingestion cells to populate `{table_fqn}` and embeddings).
	2) Optionally create the vector index (see the index DDL cell).
	3) Launch the UI with `phase = "semantic"` (see the UI cell).
	4) Ask a question (e.g., “What is the Ethereum data model?”).
- Example prompts:
	- "What is the Ethereum data model?"
	- "Where are ERC-20 transfers stored?"
	- "Summarize glossary definitions for gas and gas price."

### Docs ingestion and embeddings (Semantic setup)

First we prepare the semantic search dataset:
- Ingest docs from ../docs into table [PROJECT_ID].bq_llm.semantic_docs using PROSE_CHUNK and SQL_CHUNK. Each type chunk has different length.
- PROSE_CHUNK: Chunking(max_chars=1100, overlap=180)
- SQL_CHUNK: Chunking(max_chars=600, overlap=80)
- Controlled by RESET_TABLE_ONCE and DRY_RUN.


In [ ]:
from bigq_ai_app.utils.ingestion import (
    ingest_docs_folder,# create_embedding_model, upsert_embeddings_for_new_rows,
    PROSE_CHUNK, SQL_CHUNK
)

DOCS_DIR   = Path("..") / "docs"   # notebook sits in notebooks/
DATASET_ID = "bq_llm"              # plain dataset name only
TABLE_NAME = "semantic_docs"

# --- Run 1: initialize the table schema (DROP + CREATE), then ingest ---
RESET_TABLE_ONCE = True     # only on the very first run
DRY_RUN = False             # set to False to actually write

ingest_docs_folder(
    client=client,
    project_id=PROJECT_ID,
    location=LOCATION,
    docs_dir=DOCS_DIR,
    dataset_id=DATASET_ID,
    table_name=TABLE_NAME,
    dry_run=DRY_RUN,
    filter_filenames=["Sample-SQL-queries.md","Ethereum-blockchain-data-model.md","Ethereum-business-glossary.md"],
    prose_chunk=PROSE_CHUNK,
    sql_chunk=SQL_CHUNK,
    reset_table_once=RESET_TABLE_ONCE,     # First run does DROP+CREATE
    truncate_before_insert=False           # (ignored because we reset)
)


Simple query to verify the content length of each chunk. Here is where the fine tuning will occur depending of the type of documents to ingest.

In [ ]:
if client is None:
    raise RuntimeError("BigQuery client is not initialized. Fix auth/credentials above and re-run.")

TEST_SQL = """
SELECT
  id,
  chunk_index,
  CHAR_LENGTH(content) AS content_length
FROM
  `gen-lang-client-0962868402.bq_llm.semantic_docs`
ORDER BY
  content_length DESC
LIMIT 10;
"""
df_test = client.query(TEST_SQL).to_dataframe()
df_test




 Now we keep the schema, but make sure to truncate content before inserting 

In [ ]:
from bigq_ai_app.utils.ingestion import (
    #ingest_docs_folder, 
    create_embedding_model, upsert_embeddings_for_new_rows,
    debug_embedding_row_by_row, #new import for debugging
    PROSE_CHUNK, SQL_CHUNK
)

DOCS_DIR   = Path("..") / "docs"   # notebook sits in notebooks/
DATASET_ID = "bq_llm"              # plain dataset name only
TABLE_NAME = "semantic_docs"

# --- Runs new 2+ :---
RESET_TABLE_ONCE = False
DRY_RUN = False

import time

# --- This is your existing ingestion code ---
ingest_docs_folder(
    client=client,
    project_id=PROJECT_ID,
    location=LOCATION,
    docs_dir=DOCS_DIR,
    dataset_id=DATASET_ID,
    table_name=TABLE_NAME,
    dry_run=False,
    reset_table_once=False,
    truncate_before_insert=True
)

# --- This is your existing embedding code ---
MODEL_PATH = create_embedding_model(
    client, PROJECT_ID, LOCATION, DATASET_ID,
    model_name="text_embedding_model",
    connection_name="bq-llm-connection",
    endpoint="text-embedding-004"
)

# This should now work without the streaming buffer error
upsert_embeddings_for_new_rows(client, PROJECT_ID, LOCATION, DATASET_ID, TABLE_NAME, MODEL_PATH)

Preview rows that already have embeddings in the semantic_docs table

In [ ]:
table_fqn = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}"

# Try to detect the embedding column from the table schema
t = client.get_table(table_fqn)
embed_col = None
for f in t.schema:
	if f.mode == "REPEATED" and f.field_type.upper() in ("FLOAT", "FLOAT64"):
		embed_col = f.name
		break
# Fallback to common names if not found via schema
if embed_col is None:
	for cand in ("embedding", "embeddings", "vector", "embedding_vec"):
		if any(sf.name == cand for sf in t.schema):
			embed_col = cand
			break

if embed_col is None:
	print("No embedding-like column detected. Table schema:")
	for f in t.schema:
		print(f"  - {f.name}: {f.field_type} (mode={f.mode})")
else:
	sql = f"""
	SELECT
	  id,
	  chunk_index,
	  SUBSTR(content, 1, 160) AS content_preview,
	  ARRAY_LENGTH({embed_col}) AS embedding_dim,
	  {embed_col} AS embedding
	FROM `{table_fqn}`
	WHERE {embed_col} IS NOT NULL
	ORDER BY chunk_index
	LIMIT 8
	"""
	df_embed = client.query(sql).to_dataframe()

	# Compact preview of the embedding values (first few numbers)
	def _head(x, n=8):
		try:
			return list(x[:n])
		except Exception:
			try:
				import numpy as np  # already available in env typically; used only if present
				return list(np.array(x).flatten()[:n])
			except Exception:
				return None

	if df_embed.empty:
		counts = client.query(
			f"SELECT COUNTIF({embed_col} IS NOT NULL) AS with_embed, COUNT(*) AS total FROM `{table_fqn}`"
		).to_dataframe().iloc[0]
		print(f"No rows with embeddings yet. with_embed={counts['with_embed']}, total={counts['total']}")
	else:
		df_embed["embedding_head"] = df_embed["embedding"].apply(_head)
		preview_cols = ["id", "chunk_index", "content_preview", "embedding_dim", "embedding_head"]
		display(df_embed[preview_cols])

Vector Index will fail as there are less than 500 records in our vector database. So the error is captured.

In [ ]:
# Safely create the vector index with error handling (no raise)
index_name = "semantic_docs_index"

if client is None:
	print("BigQuery client is not initialized. Skipping vector index creation.")
elif not table_fqn or not embed_col:
	print("Table FQN or embedding column not set. Skipping vector index creation.")
else:
	ddl = f"""
	CREATE OR REPLACE VECTOR INDEX {index_name}
	ON `{table_fqn}`({embed_col})
	OPTIONS(index_type = 'IVF', distance_type = 'COSINE');
	"""
	try:
		job = client.query(ddl)
		job.result()
		print(f"Vector index ensured: {index_name} on {table_fqn} (distance=COSINE, type=IVF)")
	except Exception as e:
		print(f"Vector index creation skipped due to error: {e}")

### Retreival

Now let's retrieve some data by asking "What is the Ethereum data model?"

In [ ]:
#-- For small tables (< 5,000 rows), you don't need to create an index.
#-- Instead, use the "use_brute_force" option in VECTOR_SEARCH.

# Initialize the BigQuery client. This object will send the query to the API.
client = bigquery.Client()


# 1. Define search parameters
search_query = "What is the Ethereum data model?"
table_fqn_str = f"{PROJECT_ID}.bq_llm.semantic_docs"
model_path = f"`{PROJECT_ID}.bq_llm.text_embedding_model`"
embed_col = None  # Will be detected
top_k = 5

# Detect the embedding column from the table schema
t = client.get_table(table_fqn_str)
print(f"Table schema for {table_fqn_str}:")
for f in t.schema:
    print(f"  - {f.name}: {f.field_type} (mode={f.mode})")
schema_names = [f.name for f in t.schema]
for f in t.schema:
    if f.mode == "REPEATED" and f.field_type.upper() in ("FLOAT", "FLOAT64"):
        embed_col = f.name
        break
# Fallback to common names if not found via schema
if embed_col is None:
    for cand in ("embedding", "embeddings", "vector", "embedding_vec"):
        if cand in schema_names:
            f = next((f for f in t.schema if f.name == cand), None)
            if f and f.mode == "REPEATED" and f.field_type.upper() in ("FLOAT", "FLOAT64"):
                embed_col = cand
                break

if embed_col is None:
    print("No embedding column found.")
    raise ValueError("No embedding column found in the table schema.")

print(f"Using embedding column: {embed_col}")

# Prepare column name literals (VECTOR_SEARCH expects column names as strings)
embed_col_literal = f"'{embed_col}'"
query_col_literal = "'ml_generate_embedding_result'"

# 2. Construct the vector search query
sql_query = f"""
WITH query_table AS (
  -- Generate an embedding for our search query on the fly.
  SELECT ml_generate_embedding_result
  FROM
    ML.GENERATE_EMBEDDING(
      MODEL {model_path},
      (SELECT '{search_query}' AS content)
    )
)
-- Find the nearest neighbors using VECTOR_SEARCH.
SELECT
  search_result.base.source_file AS source_file,
  search_result.base.content AS content,
  search_result.distance AS distance
FROM
  VECTOR_SEARCH(
    TABLE `{table_fqn_str}`,
    {embed_col_literal},
    TABLE query_table,
    {query_col_literal},
    top_k => {top_k},
    distance_type => 'COSINE',
    options => '{{"use_brute_force": true}}'
  ) AS search_result
ORDER BY
  distance
"""

# 3. Execute the query and display the results
if client is None:
    raise RuntimeError("BigQuery client is not initialized.")

print(f"Performing vector search for: '{search_query}'...")
job = client.query(sql_query)
results_df = job.to_dataframe()  # Waits for completion and returns a DataFrame

print("Search complete. Top results:")
display(results_df)

Vector search with optional pre-filter on base_table. You can filter by columns like kind or source_file before searching.

In [ ]:


# 0) Ensure client exists
if client is None:
    raise RuntimeError("BigQuery client is not initialized.")

from google.cloud import bigquery

# 1) Parameters
search_query_pf = "What is the Ethereum data model?"  # your query text
project_id = PROJECT_ID
dataset_id = DATASET_ID if 'DATASET_ID' in globals() else 'bq_llm'
table_name = TABLE_NAME if 'TABLE_NAME' in globals() else 'semantic_docs'

# Set one or both filters below (leave as None/empty for no filtering)
filter_kind: str | None = None            # e.g., "glossary" or "model"
filter_source_contains: str | None = None # e.g., "Ethereum-blockchain-data-model.md"

# Use brute force for small tables; set to False to allow index usage if present
use_brute_force = True

# 2) Resolve FQN and detect embedding column
table_fqn_str = f"{project_id}.{dataset_id}.{table_name}"
model_path = f"`{project_id}.{dataset_id}.text_embedding_model`"

# Detect embedding column (ARRAY<FLOAT64>)
t = client.get_table(table_fqn_str)
embed_col = None
for f in t.schema:
    if f.mode == "REPEATED" and f.field_type.upper() in ("FLOAT", "FLOAT64"):
        embed_col = f.name
        break
if embed_col is None:
    # Fallback common names
    for cand in ("embedding", "embeddings", "vector", "embedding_vec"):
        f = next((sf for sf in t.schema if sf.name == cand), None)
        if f and f.mode == "REPEATED" and f.field_type.upper() in ("FLOAT", "FLOAT64"):
            embed_col = cand
            break
if embed_col is None:
    raise ValueError("No embedding column (ARRAY<FLOAT64>) found in table schema.")

print(f"Using embedding column: {embed_col}")

# 3) Build WHERE clause safely
def _sql_quote(s: str) -> str:
    return s.replace("'", "''")

where_parts: list[str] = []
if filter_kind:
    where_parts.append(f"kind = '{_sql_quote(filter_kind)}'")
if filter_source_contains:
    where_parts.append(f"STRPOS(source_file, '{_sql_quote(filter_source_contains)}') > 0")

where_clause = (" WHERE " + " AND ".join(where_parts)) if where_parts else ""

# 4) Prepare VECTOR_SEARCH literals
embed_col_literal = f"'{embed_col}'"
query_col_literal = "'ml_generate_embedding_result'"
options_sql_literal = "'{" + ("\"use_brute_force\": true" if use_brute_force else "") + "}'"
# If not using brute force, '{}' is acceptable; above builds '{}' when false.

# 5) Construct and run the query using base_table_query with optional WHERE
sql_query_pf = f"""
WITH query_table AS (
  SELECT ml_generate_embedding_result
  FROM ML.GENERATE_EMBEDDING(
    MODEL {model_path},
    (SELECT '{_sql_quote(search_query_pf)}' AS content)
  )
)
SELECT
  search_result.base.source_file AS source_file,
  search_result.base.kind AS kind,
  search_result.base.chunk_index AS chunk_index,
  SUBSTR(search_result.base.content, 1, 240) AS content_preview,
  search_result.distance AS distance
FROM
  VECTOR_SEARCH(
    (SELECT * FROM `{table_fqn_str}`{where_clause}),
    {embed_col_literal},
    TABLE query_table,
    {query_col_literal},
    top_k => 5,
    distance_type => 'COSINE',
    options => {options_sql_literal}
  ) AS search_result
ORDER BY distance
"""

print("Pre-filter WHERE clause:", where_clause or "<none>")
print(f"Performing vector search with pre-filter for: '{search_query_pf}'...")
job_pf = client.query(sql_query_pf)
results_pf = job_pf.to_dataframe()
print("Search complete. Top (pre-filtered) results:")
display(results_pf)

Let's see the UI and ask for example: What is the Ethereum data model?

In [ ]:
# Launch the UI with phase selection
from bigq_ai_app.ui import create_ui

# Choose phase: architect, semantic, multimodal
phase = "semantic"  # change to "semantic" or "multimodal"

demo = create_ui(phase)
demo.launch()


## The Multimodal Pioneer

### OCR PDFs with Vision + BigFrames

This section uses Google Vision OCR and handles uploaded PDFs.
- If a file is uploaded, it extracts text (Vision OCR via GCS for PDFs; local fallback for non-cloud).
- If no file is uploaded, it falls back to the semantic retrieval.

In [ ]:
# Install optional libs if needed (uncomment if missing)
# import sys
# !{sys.executable} -m uv pip install google-cloud-vision google-cloud-storage bigframes pdfplumber

from bigq_ai_app.core.multimodal_orchestrator import extract_text_from_files
from bigq_ai_app.core.config import BaseConfig

# Example usage with a local test file (replace with your path)
local_txt = "../tests/test_document.txt"
texts, notes, uris = extract_text_from_files([local_txt])
print("Notes:\n", notes)
print("Extracted text head:\n", (texts[0] or "")[:200])

# If you have a GCS bucket configured and a pdf in local path, upload+OCR
# BaseConfig.GCS_BUCKET should be set; e.g., in .env: GCS_BUCKET=my-bucket
# local_pdf = "../docs/sample.pdf"
# texts, notes, uris = extract_text_from_files([local_pdf])
# print("Vision OCR notes:\n", notes)
# print("GCS URIs:", uris)

For testing the Gradio app, you can use the image file at `../tests/original-ethereum-whitepaper-page1.png`.

In [ ]:
# Launch the UI with phase selection
from bigq_ai_app.ui import create_ui

# Choose phase: architect, semantic, multimodal
phase = "multimodal"

demo = create_ui(phase)
demo.launch()

Thank you!